# Gerador de dados
Script focada na geração de dados artificiais para o projeto Enterprise Challenge - FIAP

## Preparações iniciais

In [44]:
import pandas as pd, random, oracledb
import numpy as np
from faker import Faker
import datetime

### Definindo parametros iniciais

In [45]:
id_maquina = 101
id_potenciometro = 1
id_termistor = 2
id_mpu6050 = 3

reg_max = 50000

conn = oracledb.connect(user="rm562274", password="090402", dsn='oracle.fiap.com.br:1521/ORCL')



## Setando valores padrões

In [46]:

# Inicializa o Faker para gerar dados em português brasileiro
fake = Faker('pt_BR')

# Para garantir que os dados "aleatórios" sejam sempre os mesmos, o que é bom para testes
Faker.seed(0)
np.random.seed(0)


In [47]:

def gerar_fabricas(num_fabricas):
    print("Iniciando a geração de dados para as Dimensões...")
    fabricas = []
    for i in range(1, num_fabricas + 1):
        fabricas.append({
            'id_fabrica': i,
            'nome_fabrica': fake.company(),
            'cidade': fake.city(),
            'estado': fake.state_abbr()
        })
    df_fabricas = pd.DataFrame(fabricas)
    return(df_fabricas)


In [48]:
def gerar_sensores(num_sensores_gerados):
    # --- Dimensão Sensores ---
    tipos_sensor = {'Temperatura':'Ds18b20', 'Vibracao':'MPU-6050', 'Corrente':'3590S'}
    sensores = []
    id_sensor_counter = 1
    for tipo in tipos_sensor:
        for i in range(0, num_sensores_gerados):
            
            sensores.append({
                'id_sensor': id_sensor_counter,
                'nome_sensor': f'{tipos_sensor[tipo]}-{fake.bothify(text="???-####")}',
                'tipo_sensor': tipo,
                'fabricante_sensor': fake.company(),
                'faixa_operacional_min': '',
                'faixa_operacional_max': ''
            })
            id_sensor_counter += 1

    df_sensores = pd.DataFrame(sensores)
    df_sensores.loc[df_sensores['tipo_sensor'] == 'Temperatura', 'faixa_operacional_min'] = -55.0
    df_sensores.loc[df_sensores['tipo_sensor'] == 'Temperatura', 'faixa_operacional_max'] = 125.0
    df_sensores.loc[df_sensores['tipo_sensor'] == 'Vibracao', 'faixa_operacional_min'] = 0.0
    df_sensores.loc[df_sensores['tipo_sensor'] == 'Vibracao', 'faixa_operacional_max'] = 150.00
    df_sensores.loc[df_sensores['tipo_sensor'] == 'Corrente', 'faixa_operacional_min'] = 4
    df_sensores.loc[df_sensores['tipo_sensor'] == 'Corrente', 'faixa_operacional_max'] = 8
    return df_sensores


In [49]:
def gerar_lote(qtd_lote, frequencia, minutos_lote, data_inicial: str = "2025-08-01 08:00"):
    '''
    qtd_lote: Quantos lotes serão gerados
    frequencia: A frequencia de geração de dados de cada lote
    minutos_lote: Quantos minutos serão considerados 1 lote inteiro
    '''
    periodo = (60/frequencia)*minutos_lote
    print(periodo)
    print(int(data_inicial.split('-')[0]),int(data_inicial.split('-')[1]),int(data_inicial.split('-')[2].split(' ')[0]),int(data_inicial.split('-')[2].split(' ')[1].split(':')[0]))

    data_inicial = datetime.datetime(
        int(data_inicial.split('-')[0]),
        int(data_inicial.split('-')[1]),
        int(data_inicial.split('-')[2].split(' ')[0]),
        int(data_inicial.split('-')[2].split(' ')[1].split(':')[0]),
        int(data_inicial.split('-')[2].split(' ')[1].split(':')[1]),
        0,
        0
        )
    
    df_lote = pd.DataFrame(
        {
        'id':[],
        'timestamp':[]
        }
    )

    for a in range(1, qtd_lote+1):
        if a == 1:
            lote_temp = pd.DataFrame({'timestamp':pd.date_range(data_inicial, periods=periodo, freq=f'{int(frequencia)}min')}) 
        else:
            lote_temp = pd.DataFrame({'timestamp':pd.date_range(max(df_lote['timestamp']), periods=periodo, freq=f'{int(frequencia)}min')})
        lote_temp['id'] = a
        df_lote = pd.concat([df_lote, lote_temp[['id', 'timestamp']]], ignore_index=True)

    df_lote = df_lote.astype({'id':int})
    return df_lote


In [50]:
def gerar_maquinas(num_maquinas):
    # --- Dimensão Máquinas ---
    maquinas = []
    ids_fabricas_validos = df_fabricas['id_fabrica'].tolist()
    for i in range(1, num_maquinas + 1):
        maquinas.append({
            'id_maquina': i,
            'nome_maquina': f'WeldBot-{2000 + i}',
            'fabricante': fake.company(),
            'modelo': np.random.choice(['TX-100', 'RX-500', 'W-Master']),
            'dt_instalacao': fake.past_date(start_date='-2y'),
            'id_fabrica': np.random.choice(ids_fabricas_validos) # Garante integridade
        })
    df_maquinas = pd.DataFrame(maquinas)
    return df_maquinas


In [51]:
# print(int(len(df_sensores['id_sensor'])/3))
def gerar_sistema(qtd_sistemas):
    sistema = []

    for i in range(1, qtd_sistemas + 1):
        sistema.append({
            'id_sistema':i,
            'endereco_sistema':fake.bothify(text="SETOR-####"),
            'nome_sistema':f'ESP32-',
            # 'id_maquina':random.choice(df_maquinas['id_maquina'].tolist())
        })

    df_sistema = pd.DataFrame(sistema)
    df_sistema['nome_sistema'] = df_sistema['nome_sistema'] + df_sistema['endereco_sistema']
    return df_sistema

# df_sistema = gerar_sistema()
# # df_sistema['id_maquina'] = random.choice(df_maquinas['id_maquina'].tolist())
# display(df_sistema)

In [ ]:
df_fabricas = gerar_fabricas(6)
cursor = conn.cursor()


In [53]:
df_sensores = gerar_sensores(4)
display(df_sensores)


,id_sensor,nome_sensor,tipo_sensor,fabricante_sensor,faixa_operacional_min,faixa_operacional_max
0,1,Ds18b20-GFz-6604,Temperatura,Mendes,-55.0,125.0
1,2,Ds18b20-isi-5938,Temperatura,Pinto Ltda.,-55.0,125.0
2,3,Ds18b20-gUe-8924,Temperatura,Melo,-55.0,125.0
3,4,Ds18b20-uNO-8156,Temperatura,Novaes Mendes - ME,-55.0,125.0
4,5,MPU-6050-afU-8408,Vibracao,Santos,0.0,150.0
5,6,MPU-6050-pUu-0975,Vibracao,Correia - EI,0.0,150.0
6,7,MPU-6050-Cff-3328,Vibracao,Moraes,0.0,150.0
7,8,MPU-6050-sTh-7148,Vibracao,Nogueira,0.0,150.0
8,9,3590S-sCf-3989,Corrente,Dias,4,8
9,10,3590S-mlc-9342,Corrente,Melo,4,8


In [ ]:
df_sistema = gerar_sistema(int(len(df_sensores['id_sensor'])/3))
df_sistema['id_maquina'] = [int(a) for a in df_sistema['id_sistema']]
display(df_sistema)

df_sensores.loc[df_sensores['tipo_sensor'] == "Temperatura", "id_sistema"] = [int(a) for a in df_sistema['id_sistema']]
df_sensores.loc[df_sensores['tipo_sensor'] == "Vibracao", "id_sistema"] = [int(a) for a in df_sistema['id_sistema']]
df_sensores.loc[df_sensores['tipo_sensor'] == "Corrente", "id_sistema"] = [int(a) for a in df_sistema['id_sistema']]
df_sensores['id_sistema'] = df_sensores['id_sistema'].astype(int) 

display(df_sensores)

,id_sistema,endereco_sistema,nome_sistema,id_maquina
0,1,SETOR-2518,ESP32-SETOR-2518,1
1,2,SETOR-5367,ESP32-SETOR-5367,2
2,3,SETOR-1097,ESP32-SETOR-1097,3
3,4,SETOR-9519,ESP32-SETOR-9519,4


In [13]:
df_lote = gerar_lote(5, 60, 1)
display(df_lote)


1.0
2025 8 1 8


C:\Users\Davi\AppData\Local\Temp\ipykernel_23940\136447155.py:30: FutureWarning: Non-integer 'periods' in pd.date_range, pd.timedelta_range, pd.period_range, and pd.interval_range are deprecated and will raise in a future version.
  lote_temp = pd.DataFrame({'timestamp':pd.date_range(data_inicial, periods=periodo, freq=f'{int(frequencia)}min')})
C:\Users\Davi\AppData\Local\Temp\ipykernel_23940\136447155.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_lote = pd.concat([df_lote, lote_temp[['id', 'timestamp']]], ignore_index=True)
C:\Users\Davi\AppData\Local\Temp\ipykernel_23940\136447155.py:32: FutureWarning: Non-integer 'periods' in pd.date_range, pd.timedelta_range, pd.period_range, and pd.interval_range are deprecated and will rais

,id,timestamp
0,1,2025-08-01 08:00:00
1,2,2025-08-01 08:00:00
2,3,2025-08-01 08:00:00
3,4,2025-08-01 08:00:00
4,5,2025-08-01 08:00:00


In [ ]:
# df_maquinas = gerar_maquinas(len(df_fabricas))

display(df_maquinas[:4])


,id_maquina,nome_maquina,fabricante,modelo,dt_instalacao,id_fabrica
0,1,WeldBot-2001,Cassiano Rezende - ME,RX-500,2024-07-15,1
1,2,WeldBot-2002,Moura,RX-500,2024-10-25,2
2,3,WeldBot-2003,Oliveira S/A,TX-100,2025-03-11,3
3,4,WeldBot-2004,Souza - ME,W-Master,2025-09-03,4


In [99]:
def gerar_fatos(reg: int = 5000):
    sistema = pd.read_sql('SELECT * FROM DM_SISTEMAS', conn)
    maquinario = pd.read_sql('SELECT * FROM DM_MAQUINA', conn)
    print("\nIniciando a geração de dados para as Tabelas Fato...")

    # Extrai listas de IDs válidos das dimensões
    ids_maquinas_validos = maquinario['ID_MAQUINA'].tolist()
    ids_sistema_validos = sistema['ID_SISTEMA'].tolist()
    print(ids_maquinas_validos, ids_sistema_validos)
    # # --- Fato Registros de Sensores ---
    registros = {
        'id_registro': range(1, reg + 1),
        'id_sistema': np.random.choice(ids_sistema_validos, reg),
        'id_maquina': np.random.choice(ids_maquinas_validos, reg),
        'timestamp_registro': pd.to_datetime(np.linspace(
            datetime.datetime(2025, 8, 1).timestamp(),
            datetime.datetime.now().timestamp(),
            reg), unit='s'),
        'vl_temperatura': np.random.normal(80, 15, reg),
        'vl_vibracao': np.random.normal(4, 2, reg),
        'vl_corrente': np.random.normal(150, 5, reg)
    }
    df_registros = pd.DataFrame(registros)

    # df_registros = pd.DataFrame(registros)
    return df_registros
    # display(df_registros)
df_fato = gerar_fatos(reg = 1)
display(df_fato)
# df_fato.to_excel(r'C:\Users\Davi\Documents\Projetos\FIAP\FASE 6\Enterprise Challenge Fase 4\docs\estrutura de teste\Dados_simulados.xlsx', index=False)


Iniciando a geração de dados para as Tabelas Fato...
[1, 2, 3, 4] [1, 2, 3, 4]


C:\Users\Davi\AppData\Local\Temp\ipykernel_23940\3798535030.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sistema = pd.read_sql('SELECT * FROM DM_SISTEMAS', conn)
C:\Users\Davi\AppData\Local\Temp\ipykernel_23940\3798535030.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  maquinario = pd.read_sql('SELECT * FROM DM_MAQUINA', conn)


,id_registro,id_sistema,id_maquina,timestamp_registro,vl_temperatura,vl_vibracao,vl_corrente
0,1,1,2,2025-08-01 03:00:00,89.095835,4.003522,145.049199
